In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
from matplotlib import pyplot as plt

C:\Users\firda\AppData\Local\Temp\ipykernel_4848\3513348859.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\firda\AppData\Local\Temp\ipykernel_4848\3513348859.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
filtered_df = spacex_df[spacex_df['Launch Site']=='CCAFS LC-40'].groupby(['Launch Site','class'],as_index=False). \
           count()
filtered_df.columns = filtered_df.columns.str.replace('Unnamed: 0','Count')
filtered_df

,Launch Site,class,Count,Flight Number,Payload Mass (kg),Booster Version,Booster Version Category
0,CCAFS LC-40,0,19,19,19,19,19
1,CCAFS LC-40,1,7,7,7,7,7


In [3]:
# Create a dash application
app = dash.Dash(__name__)

In [4]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Div([
                                    # Add an division
                                    html.Div([
                                        # Create an division for adding dropdown helper text for report type
                                        html.Div(
                                            [
                                            html.H2('Launch Sites:', style={'margin-right': '2em'}),
                                            ]
                                        ),
dcc.Dropdown(
                          id='site_dropdown',

                      options=[
                           {'label': 'All Sites', 'value': 'All Sites'},
                          {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                          {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                          {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                          
      ],
      value='All Sites',
      placeholder="All Sites",
    #Searchable=True,
    style={'textAlign': 'center', 'width': '80%', 'font-size': 24, 'padding':'3px'}
      ),
                                html.Br(),
                                html.Div(
                                            [
                                            html.H2('Success pie chart:', style={'felxible': '2em'}),
                                            ]),        

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                html.Div(dcc.Graph( id='success_pie_chart')),
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                #html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                               # html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                               html.Div(
                                            [
                                            html.H2('Select Payload Range :', style={'margin-right': '2em'}),
                                            ]),         
                                        
dcc.RangeSlider(
                               min=0, max= 10000, step= 1000,
                               value=[min_payload,max_payload],
                                id='payload_slider'),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success_payload_scatter_chart')),
                                ]),
                                ]),
                                # Add Computed graphs
                                # REVIEW3: Observe how we add an empty division and providing an id that will be updated during callback
                                
                                ])
                               
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success_pie_chart', component_property='figure'),
                Input(component_id='site_dropdown', component_property='value'))
        
        # return the outcomes piechart for a selected site
def get_pie_chart(value):
    filtered_df = spacex_df[spacex_df['Launch Site']==value].groupby(['Launch Site','class'],as_index=False). \
           count()
    filtered_df.columns = filtered_df.columns.str.replace('Unnamed: 0','Count')


    if value == 'All Sites':
        filtered_df = spacex_df[spacex_df['class']==1].groupby(['Launch Site'],as_index=False). \
       count()
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Total successful launch for '+str(value))
        return fig
    else:
        
        fig = px.pie(filtered_df, values='Count',
        names='class', 
        title='Total successful launch for '+str(value))
        return fig

        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success_payload_scatter_chart', component_property='figure'),
              [Input(component_id='site_dropdown', component_property='value'),
              Input(component_id='payload_slider', component_property='value')])

def success_payload_scatter_chart(site_dropdown,payload_slider):

    if site_dropdown == 'All Sites':
        filtered_df = spacex_df
    else:
        
        filtered_df = spacex_df[spacex_df['Launch Site']==site_dropdown]

    filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] <= payload_slider[1]) & (filtered_df['Payload Mass (kg)'] >= payload_slider[0])]        
        

    fig1 = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',color='Booster Version Category', title='Correlation between payload and success for '+str(site_dropdown))
    return fig1
    


get_pie_chart('Launch Site')
plt.show()

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Oct/2022 12:32:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2022 12:32:28] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2022 12:32:28] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2022 12:32:28] "GET /_favicon.ico?v=2.6.2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2022 12:32:28] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2022 12:32:28] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2022 12:32:28] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2022 12:32:28] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2022 12:32:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2022 12:32:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2022 12:32:44] "POST /_dash-upd